In [8]:
# Datafetch
# !wget ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE129nnn/GSE129788/suppl/GSE129788_RAW.tar
# !tar -xvf GSE129788_RAW.tar
# !gunzip *.gz

GSM3722100_YX1L_10X.txt.gz
GSM3722101_YX2L_10X.txt.gz
GSM3722102_YX3R_10X.txt.gz
GSM3722103_YX4R_10X.txt.gz
GSM3722104_YX5R_10X.txt.gz
GSM3722105_YX6L_10X.txt.gz
GSM3722106_YX7R_10X.txt.gz
GSM3722107_YX8L_10X.txt.gz
GSM3722108_OX1X_10X.txt.gz
GSM3722109_OX2X_10X.txt.gz
GSM3722110_OX3X_10X.txt.gz
GSM3722111_OX4X_10X.txt.gz
GSM3722112_OX5X_10X.txt.gz
GSM3722113_OX6X_10X.txt.gz
GSM3722114_OX7X_10X.txt.gz
GSM3722115_OX8X_10X.txt.gz


In [9]:
!mkdir aging_brain
!mv GSM37221* aging_brain
!mv GSE129788_RAW.tar aging_brain/

mkdir: cannot create directory ‘aging_brain’: File exists
mv: cannot stat 'GSM37221*': No such file or directory


In [25]:
sample_locations = !ls ./aging_brain/GSM*

In [28]:
import numpy as np
import matplotlib.pyplot as plt
import scanpy as sc

samples = []

for sample_location in sample_locations:
    samples.append(sc.read(sample_location))

In [30]:
samples = [s.T for s in samples]

In [35]:
# Samples are already normalized so we will have to stack them unless I want to spend forever and a day downloading bullshit raw data

samples

[AnnData object with n_obs × n_vars = 1370 × 14699 ,
 AnnData object with n_obs × n_vars = 1666 × 14699 ,
 AnnData object with n_obs × n_vars = 1851 × 14699 ,
 AnnData object with n_obs × n_vars = 2279 × 14699 ,
 AnnData object with n_obs × n_vars = 1856 × 14699 ,
 AnnData object with n_obs × n_vars = 1839 × 14699 ,
 AnnData object with n_obs × n_vars = 2519 × 14699 ,
 AnnData object with n_obs × n_vars = 2648 × 14699 ,
 AnnData object with n_obs × n_vars = 1823 × 14699 ,
 AnnData object with n_obs × n_vars = 1489 × 14699 ,
 AnnData object with n_obs × n_vars = 3984 × 14699 ,
 AnnData object with n_obs × n_vars = 3803 × 14699 ,
 AnnData object with n_obs × n_vars = 2175 × 14699 ,
 AnnData object with n_obs × n_vars = 2242 × 14699 ,
 AnnData object with n_obs × n_vars = 2597 × 14699 ,
 AnnData object with n_obs × n_vars = 2928 × 14699 ]

In [37]:
combined = np.zeros((0,14699))

for sample in samples:
    combined = np.vstack([combined,sample.X])
    
combined

array([[0.        , 1.34027112, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [41]:
stacked = sc.AnnData(combined)
stacked

AnnData object with n_obs × n_vars = 37069 × 14699 

In [46]:
filter_result = sc.pp.filter_genes_dispersion(  # select highly-variable genes
    stacked.X, flavor='cell_ranger', n_top_genes=2000, log=True
)
filtered = stacked[:, filter_result.gene_subset]     # subset the genes


In [47]:
filtered

View of AnnData object with n_obs × n_vars = 37069 × 1999 

In [51]:
# We have to set up young vs old annotation:

young = np.zeros(37069,dtype=bool)
old = np.zeros(37069,dtype=bool)

young_samples = np.sum([s.shape[0] for s in samples[:8]])
young[:young_samples] = True

old_samples = np.sum([s.shape[0] for s in samples[8:]])
old[old_samples:] = True

In [53]:
young_samples+old_samples

37069